In [1]:
import numpy as np
import pandas as pd 
import statistics

In [2]:
df_16 = pd.read_csv("2016.csv", encoding = "windows-1254")
df_17 = pd.read_csv("2017.csv", encoding = "windows-1254")
df_18 = pd.read_csv("2018.csv", encoding = "windows-1254")
df_19 = pd.read_csv("2019.csv", encoding = "windows-1254")
df_20 = pd.read_csv("2020.csv", encoding = "windows-1254")

In [3]:
df_16.head()

,Tarih,Saat,Tüketim Miktarı (MWh),dec
0,01.01.2016,00:00,26.277,24
1,01.01.2016,01:00,24.991,82
2,01.01.2016,02:00,23.532,61
3,01.01.2016,03:00,22.464,78
4,01.01.2016,04:00,22.002,91


In [4]:
## Preparation for data
def df_proces (X):
    X["Tüketim Miktarı (MWh)"] = X["Tüketim Miktarı (MWh)"] * 1000
    X["Tüketim Miktarı (MWh)"] = X["Tüketim Miktarı (MWh)"] + X["dec"] / 100
    X.drop(columns = ["dec"], inplace = True)
    return X

In [5]:
data = [df_16, df_17, df_18, df_19, df_20]

In [6]:
for i in data: 
    df_proces(i)
df = pd.concat(data, ignore_index = True)
df.rename(columns = {"Tüketim Miktarı (MWh)" : "mwh"}, inplace = True)

In [7]:
df_16.head()

,Tarih,Saat,Tüketim Miktarı (MWh)
0,01.01.2016,00:00,26277.24
1,01.01.2016,01:00,24991.82
2,01.01.2016,02:00,23532.61
3,01.01.2016,03:00,22464.78
4,01.01.2016,04:00,22002.91


In [8]:
df["Tarih"] = pd.to_datetime(df["Tarih"], dayfirst = True)

In [9]:
## Creating lag values
df["lag_48"] = df["mwh"].shift(48)
df["lag_168"] = df["mwh"].shift(168)

In [29]:
## Creating test data
test_df = df[df["Tarih"]>= "2020-11-01"]

In [11]:
def mape(y_act, y_pred):
    return np.mean(np.abs((y_act - y_pred) / y_act)) * 100

In [12]:
## Because of the zero division error, I should check tha data
df[df["mwh"]==0]

,Tarih,Saat,mwh,lag_48,lag_168
2066,2016-03-27,02:00,0.0,25524.5,24546.43


In [13]:
df.drop(index = [2066], axis = 0, inplace = True)

In [14]:
mape(test_df["mwh"], test_df["lag_48"])

7.789204389494431

In [15]:
mape(test_df["mwh"], test_df["lag_168"])

3.4536690170083504

In [16]:
df.reset_index(inplace = True, drop = True)


As it can be seen from the mape results, using lag_168 values as regressors, is more successful than the lag_48 values. Before testing the results, intuitively using the last week’s same day and same hour values (lag_168) for predicting next day consumption is more sensible than two days before the values. The results confirmed intuition. 

## Part B

In [17]:
from sklearn.linear_model import LinearRegression

In [18]:
## I want to use hour values as a variable for linear regression so I convert the hour data as a number.
x = []
for i in range(len(df)):
    x.append(df["Saat"][i][:-3])
df["Saat"] = x

In [21]:
df["Saat"] = pd.to_numeric(df["Saat"])

In [22]:
df.head()

,Tarih,Saat,mwh,lag_48,lag_168
0,2016-01-01,0,26277.24,NaN,NaN
1,2016-01-01,1,24991.82,NaN,NaN
2,2016-01-01,2,23532.61,NaN,NaN
3,2016-01-01,3,22464.78,NaN,NaN
4,2016-01-01,4,22002.91,NaN,NaN


In [31]:
train_df = df.dropna()
train_df = train_df[train_df["Tarih"]< "2020-11-01"]

In [32]:
x_train = train_df.drop(["mwh","Tarih"], axis = 1)
y_train = train_df["mwh"]

In [33]:
y_test = test_df["mwh"]
x_test = test_df.drop(["mwh","Tarih"], axis = 1)

In [34]:
model = LinearRegression()

In [35]:
model.fit(x_train,y_train)
y_pred = model.predict(x_test)

In [36]:
mape(y_test, y_pred)

4.129599053606711

- MAPE Lag 48 = 7.789  
- MAPE Lag 168 = 3.453  
- MAPE Linear Regression = 4.129

Linear Regression model is better than lag 48 regressor but it still get behind the lag 168 regressor.

## Part C

In [37]:
def lin_reg (x_train, y_train, x_test, y_test):
    model = LinearRegression()
    model.fit(x_train,y_train)
    yy_pred = model.predict(x_test)
    mape_res = mape(y_test, yy_pred)
    coef = model.coef_
    return yy_pred, mape_res, coef

In [38]:
mapes = []
for i in range(24):
    temp_train = train_df[train_df["Saat"] == i]
    tempy_train = temp_train["mwh"]
    tempx_train = temp_train.drop(["mwh","Tarih"], axis = 1)
    temp_test = test_df[test_df["Saat"] == i]
    tempy_test = temp_test["mwh"]
    tempx_test = temp_test.drop(["mwh","Tarih"], axis = 1)
    k = lin_reg (tempx_train, tempy_train, tempx_test, tempy_test)
    mapes.append(k[1])
    print("------------------------------------------")
    print("Hour :", i)
    print("Hourly Mape:", i , " = ", k[1])
    print("Coefficients :", i, k[2][1:3])
    

------------------------------------------
Hour : 0
Hourly Mape: 0  =  3.2668954330862214
Coefficients : 0 [0.48688307 0.44283593]
------------------------------------------
Hour : 1
Hourly Mape: 1  =  3.269987276090032
Coefficients : 1 [0.49195523 0.43642319]
------------------------------------------
Hour : 2
Hourly Mape: 2  =  3.3880215216792435
Coefficients : 2 [0.47390162 0.41952495]
------------------------------------------
Hour : 3
Hourly Mape: 3  =  3.1903589208889556
Coefficients : 3 [0.49279757 0.42695905]
------------------------------------------
Hour : 4
Hourly Mape: 4  =  3.1680997634309196
Coefficients : 4 [0.47468522 0.44493902]
------------------------------------------
Hour : 5
Hourly Mape: 5  =  3.1566336972729077
Coefficients : 5 [0.43984866 0.47845986]
------------------------------------------
Hour : 6
Hourly Mape: 6  =  3.1313439692798304
Coefficients : 6 [0.37596355 0.5462    ]
------------------------------------------
Hour : 7
Hourly Mape: 7  =  3.74744788587

In [39]:
for i in range(len(mapes)):
    print("Hour", i, "=", round(mapes[i], 3))

Hour 0 = 3.267
Hour 1 = 3.27
Hour 2 = 3.388
Hour 3 = 3.19
Hour 4 = 3.168
Hour 5 = 3.157
Hour 6 = 3.131
Hour 7 = 3.747
Hour 8 = 4.59
Hour 9 = 5.3
Hour 10 = 5.842
Hour 11 = 5.99
Hour 12 = 6.233
Hour 13 = 6.355
Hour 14 = 6.19
Hour 15 = 5.846
Hour 16 = 4.968
Hour 17 = 4.506
Hour 18 = 3.677
Hour 19 = 3.382
Hour 20 = 3.117
Hour 21 = 3.111
Hour 22 = 3.126
Hour 23 = 3.356


In [40]:
statistics.mean(mapes)

4.246242302041696

In [41]:
# 00:00 - 07:00 average mape value
statistics.mean(mapes[0:8])

3.2898485584497843

In [42]:
# 08:00 - 17:00 average mape value
statistics.mean(mapes[8:16])

5.793274098164193

In [43]:
# 17:00 - 24:00 average mape value
statistics.mean(mapes[16:24])

3.655604249511112

- MAPE Lag 48 = 7.789  
- MAPE Lag 168 = 3.453  
- MAPE Linear Regression = 4.129
- MAPE Linear Regression(hourly prediction average) = 4.246
- MAPE Linear Regression (00:00 - 07:00 hours)  = 3.289
- MAPE Linear Regression (08:00 - 17:00 hours)  = 5.793
- MAPE Linear Regression (17:00 - 24:00 hours)  = 3.655

Hourly prediction looks successful between 00:00 - 07:00 and 17:00 - 24:00 hours, but avarege mape value for all day is slightly worse than part b model and it still get behind the lag_168 regressor.

## Task D

In [60]:
df2 = df.copy()

In [64]:
wide_df = df2.pivot_table(index='Tarih', columns='Saat', values=["lag_48","lag_168","mwh"])
wide_df.dropna(axis = 0, how = "any", inplace = True)


In [66]:
wide_df.tail()

lag_168                                                    \
Saat              0         1         2         3         4         5    
Tarih                                                                    
2020-11-26  33455.97  31805.17  30698.22  29846.76  29600.96  29899.31   
2020-11-27  33651.81  31949.53  30832.30  29991.21  29795.53  29951.31   
2020-11-28  33697.36  31916.72  30692.47  29828.03  29425.40  29748.05   
2020-11-29  31685.68  30105.78  28703.45  28072.20  27651.60  27566.19   
2020-11-30  30426.00  29097.77  28176.27  27599.20  27238.99  27754.59   

                                                                        \
Saat              6         7         8         9         10        11   
Tarih                                                                    
2020-11-26  30743.88  32074.70  35610.62  37711.29  38233.15  38704.74   
2020-11-27  30758.76  32370.10  36386.47  39029.49  39834.37  40475.03   
2020-11-28  30186.14  30564.89  32613.91  34058.84  34969.54  35792.39   
2020-11-29  27737.26  27439.05  26905.24  27247.25  27974.74  28525.59   
2020-11-30  28747.47  30989.04  35068.99  36907.37  37012.71  36950.05   

                                                                        \
Saat              12        13        14        15        16        17   
Tarih                                                                    
2020-11-26  37415.32  38196.98  39412.98  39853.82  40918.91  42208.11   
2020-11-27  38482.23  39003.06  40727.71  40550.85  40976.32  41854.37   
2020-11-28  34480.23  35028.71  35312.45  35408.44  36581.59  38630.06   
2020-11-29  28433.65  28962.66  29334.07  30301.50  32108.31  34919.68   
2020-11-30  35296.28  35986.82  37089.41  37586.08  39435.91  41569.99   

                                                                        \
Saat              18        19        20        21        22        23   
Tarih                                                                    
2020-11-26  41673.45  40553.66  39499.32  38429.24  37270.44  35731.83   
2020-11-27  41564.30  40221.79  38875.45  37749.83  36786.90  35102.41   
2020-11-28  39379.15  38432.76  37293.60  36304.68  35250.20  33817.76   
2020-11-29  36647.32  35999.47  35183.37  34230.51  33602.91  32178.03   
2020-11-30  41710.94  40297.34  38951.67  37856.67  36983.95  35594.03   

              lag_48                                                    \
Saat              0         1         2         3         4         5    
Tarih                                                                    
2020-11-26  33520.81  31518.27  30513.78  29844.10  29650.85  29873.42   
2020-11-27  33585.86  32175.55  31064.86  30213.43  29958.96  30234.26   
2020-11-28  33902.71  32459.36  31023.35  30190.73  30055.35  30335.29   
2020-11-29  33896.52  31841.72  30897.69  30191.20  29956.10  30199.97   
2020-11-30  33935.56  31919.44  30439.27  30020.41  29825.63  29984.86   

                                                                        \
Saat              6         7         8         9         10        11   
Tarih                                                                    
2020-11-26  30934.51  32239.30  35946.87  37504.70  37515.08  37261.41   
2020-11-27  31138.64  32838.39  36634.41  38845.22  39507.26  39896.53   
2020-11-28  31201.79  32775.21  36882.21  39059.09  39295.57  39312.61   
2020-11-29  31015.63  32518.79  36129.99  37851.53  37808.82  37668.32   
2020-11-30  30444.97  31159.67  33325.74  34868.28  35457.72  35822.08   

                                                                        \
Saat              12        13        14        15        16        17   
Tarih                                                                    
2020-11-26  35845.55  36583.20  37830.17  38656.92  40396.69  42486.34   
2020-11-27  38446.53  39059.04  40148.58  40330.14  41279.40  42837.79   
2020-11-28  37508.66  38124.78  39220.21  39460.82  40532.72  42347.17   
2020-11-29 

In [68]:
wide_df.reset_index(inplace = True)

In [82]:
wide_train = wide_df[wide_df["Tarih"] < "2020-11-01"]
wide_train_x = wide_train[["lag_168", "lag_48"]]
wide_train_y = wide_train["mwh"]

In [83]:
wide_test = wide_df[wide_df["Tarih"] >= "2020-11-01"]
wide_test_x = wide_test[["lag_168", "lag_48"]]
wide_test_y = wide_test["mwh"]

In [77]:
from sklearn.linear_model import LassoCV

In [102]:
def lasso_cv (x_train, y_train, x_test, y_test):
    model = LassoCV(cv=10, random_state=42, max_iter = 10000, n_jobs = -1)
    model.fit(x_train,y_train)
    yy_pred = model.predict(x_test)
    mape_res = mape(y_test, yy_pred)
    alpha = model.alpha_
    return yy_pred, mape_res, alpha

In [103]:
mapes_lasso = []
for i in range(24):
    tempy_train = wide_train_y[i]
    tempx_train = wide_train_x
    tempy_test = wide_test_y[i]
    tempx_test = wide_test_x
    k = lasso_cv (tempx_train, tempy_train, tempx_test, tempy_test)
    mapes_lasso.append(k[1])
    print("------------------------------------------")
    print("Hour :", i)
    print("Hourly Mape:", i , " = ", k[1])
    print("Choosen Alpha Value :", k[2])
    

------------------------------------------
Hour : 0
Hourly Mape: 0  =  1.3846133780238932
Choosen Alpha Value : 12431.332717836012
------------------------------------------
Hour : 1
Hourly Mape: 1  =  1.481389413548434
Choosen Alpha Value : 9632.3610763365
------------------------------------------
Hour : 2
Hourly Mape: 2  =  1.4585815390954726
Choosen Alpha Value : 9043.351109292304
------------------------------------------
Hour : 3
Hourly Mape: 3  =  1.3412462856253229
Choosen Alpha Value : 8654.95826046391
------------------------------------------
Hour : 4
Hourly Mape: 4  =  1.3832583640359237
Choosen Alpha Value : 8411.757499442334
------------------------------------------
Hour : 5
Hourly Mape: 5  =  1.3909375211860535
Choosen Alpha Value : 8070.515998479513
------------------------------------------
Hour : 6
Hourly Mape: 6  =  1.6965215217321317
Choosen Alpha Value : 8464.175808579437
------------------------------------------
Hour : 7
Hourly Mape: 7  =  1.904685452544801
Choo

In [111]:
for i in range(len(mapes_lasso)):
    print("Hour", i, "=", round(mapes_lasso[i], 3))

Hour 0 = 1.385
Hour 1 = 1.481
Hour 2 = 1.459
Hour 3 = 1.341
Hour 4 = 1.383
Hour 5 = 1.391
Hour 6 = 1.697
Hour 7 = 1.905
Hour 8 = 2.551
Hour 9 = 3.566
Hour 10 = 4.156
Hour 11 = 4.354
Hour 12 = 4.559
Hour 13 = 4.432
Hour 14 = 4.281
Hour 15 = 3.826
Hour 16 = 2.951
Hour 17 = 2.083
Hour 18 = 1.621
Hour 19 = 1.596
Hour 20 = 1.709
Hour 21 = 1.725
Hour 22 = 1.556
Hour 23 = 1.803


In [112]:
statistics.mean(mapes_lasso)

2.4504137869783587

In [113]:
# 00:00 - 07:00 average mape value
statistics.mean(mapes_lasso[0:8])

1.5051541844740042

In [114]:
# 08:00 - 17:00 average mape value
statistics.mean(mapes_lasso[8:16])

3.9655426041381596

In [115]:
# 17:00 - 24:00 average mape value
statistics.mean(mapes_lasso[16:24])

1.8805445723229115

- MAPE Lag 48 = 7.789  
- MAPE Lag 168 = 3.453  
- MAPE Linear Regression = 4.129
- MAPE Linear Regression(hourly prediction average) = 4.246
- MAPE Lasso Regression with L1 Penalty (hourly prediction average) = 2.450
- MAPE Lasso Regression (00:00 - 07:00 hours)  = 1.505
- MAPE Lasso Regression (08:00 - 17:00 hours)  = 3.965
- MAPE Lasso Regression (17:00 - 24:00 hours)  = 1.880



Lasso regression model looks like best predictor for this task, its mape value is the lowest one. 